In [1]:
import os
import numpy as np
import pandas as pd
import librosa
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [2]:
# Paths
DATA_DIR = "voxforge_wav_files"  # Directory containing .wav files
CSV_FILE = "gender_labels.csv"  # CSV mapping .wav files to gender


In [3]:
df = pd.read_csv("gender_labels_fixed.csv")

# Load CSV
df = df.dropna()  # Remove any missing values
# df["wav_file"] = df["wav_file"].apply(lambda x: os.path.join(DATA_DIR+'/wav/' ,os.path.basename(x)))


In [4]:
df

,speaker_id,gender,wav_file
0,1028-20100710-hne,male,voxforge_wav_files\1028-20100710-hne\wav\ar-05...
1,1028-20100710-hne,male,voxforge_wav_files\1028-20100710-hne\wav\ar-01...
2,1028-20100710-hne,male,voxforge_wav_files\1028-20100710-hne\wav\ar-07...
3,1028-20100710-hne,male,voxforge_wav_files\1028-20100710-hne\wav\ar-02...
4,1028-20100710-hne,male,voxforge_wav_files\1028-20100710-hne\wav\rp-29...
...,...,...,...
87416,zlp-20100110-wwn,male,voxforge_wav_files\zlp-20100110-wwn\wav\b0209.wav
87417,zlp-20100110-wwn,male,voxforge_wav_files\zlp-20100110-wwn\wav\b0206.wav
87418,zlp-20100110-wwn,male,voxforge_wav_files\zlp-20100110-wwn\wav\b0210.wav
87419,zlp-20100110-wwn,male,voxforge_wav_files\zlp-20100110-wwn\wav\b0204.wav


In [5]:
df["wav_file"].head()

0    voxforge_wav_files\1028-20100710-hne\wav\ar-05...
1    voxforge_wav_files\1028-20100710-hne\wav\ar-01...
2    voxforge_wav_files\1028-20100710-hne\wav\ar-07...
3    voxforge_wav_files\1028-20100710-hne\wav\ar-02...
4    voxforge_wav_files\1028-20100710-hne\wav\rp-29...
Name: wav_file, dtype: object

In [6]:
# Encode gender labels
label_encoder = LabelEncoder()
df["gender_encoded"] = label_encoder.fit_transform(df["gender"])  # male=0, female=1

In [7]:
# Function to extract MFCC features
def extract_features(file_path, max_pad_len=50):
    y, sr = librosa.load(file_path, sr=16000)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    pad_width = max_pad_len - mfccs.shape[1]
    if pad_width > 0:
        mfccs = np.pad(mfccs, ((0, 0), (0, pad_width)), mode='constant')
    else:
        mfccs = mfccs[:, :max_pad_len]
    return np.mean(mfccs, axis=1)  # Flattened feature vector

In [8]:
# Extract features and labels
X, y = [], []
for _, row in df.iterrows():
    file_path = row["wav_file"]
    print(file_path, os.path.exists(file_path))  # Debugging

    if os.path.exists(file_path):
        features = extract_features(file_path)
#         print(features)
        X.append(features)
        y.append(row["gender_encoded"])




voxforge_wav_files\1028-20100710-hne\wav\ar-05.wav True
voxforge_wav_files\1028-20100710-hne\wav\ar-01.wav True
voxforge_wav_files\1028-20100710-hne\wav\ar-07.wav True
voxforge_wav_files\1028-20100710-hne\wav\ar-02.wav True
voxforge_wav_files\1028-20100710-hne\wav\rp-29.wav True
voxforge_wav_files\1028-20100710-hne\wav\ar-03.wav True
voxforge_wav_files\1028-20100710-hne\wav\rp-30.wav True
voxforge_wav_files\1028-20100710-hne\wav\ar-06.wav True
voxforge_wav_files\1028-20100710-hne\wav\rp-31.wav True
voxforge_wav_files\1028-20100710-hne\wav\ar-04.wav True
voxforge_wav_files\1337ad-20170321-ajg\wav\en-0533.wav True
voxforge_wav_files\1337ad-20170321-ajg\wav\en-0538.wav True
voxforge_wav_files\1337ad-20170321-ajg\wav\en-0534.wav True
voxforge_wav_files\1337ad-20170321-ajg\wav\en-0540.wav True
voxforge_wav_files\1337ad-20170321-ajg\wav\en-0539.wav True
voxforge_wav_files\1337ad-20170321-ajg\wav\en-0532.wav True
voxforge_wav_files\1337ad-20170321-ajg\wav\en-0535.wav True
voxforge_wav_files\1

voxforge_wav_files\Aaron-20130527-giy\wav\b0353.wav True
voxforge_wav_files\Aaron-20130527-giy\wav\b0356.wav True
voxforge_wav_files\AbdulMoiz-20121010-juf\wav\a0295.wav True
voxforge_wav_files\AbdulMoiz-20121010-juf\wav\a0286.wav True
voxforge_wav_files\AbdulMoiz-20121010-juf\wav\a0287.wav True
voxforge_wav_files\AbdulMoiz-20121010-juf\wav\a0289.wav True
voxforge_wav_files\AbdulMoiz-20121010-juf\wav\a0292.wav True
voxforge_wav_files\AbdulMoiz-20121010-juf\wav\a0293.wav True
voxforge_wav_files\AbdulMoiz-20121010-juf\wav\a0290.wav True
voxforge_wav_files\AbdulMoiz-20121010-juf\wav\a0291.wav True
voxforge_wav_files\AbdulMoiz-20121010-juf\wav\a0288.wav True
voxforge_wav_files\AbdulMoiz-20121010-juf\wav\a0294.wav True
voxforge_wav_files\Adminvox-05232006\wav\a0446.wav True
voxforge_wav_files\Adminvox-05232006\wav\a0456.wav True
voxforge_wav_files\Adminvox-05232006\wav\a0451.wav True
voxforge_wav_files\Adminvox-05232006\wav\a0442.wav True
voxforge_wav_files\Adminvox-05232006\wav\a0445.wav T

voxforge_wav_files\AlexanderHenningDrachmann-20140108-klh\wav\a0400.wav True
voxforge_wav_files\AlexanderHenningDrachmann-20140108-klh\wav\a0398.wav True
voxforge_wav_files\AlexanderHenningDrachmann-20140108-koe\wav\b0461.wav True
voxforge_wav_files\AlexanderHenningDrachmann-20140108-koe\wav\b0459.wav True
voxforge_wav_files\AlexanderHenningDrachmann-20140108-koe\wav\b0458.wav True
voxforge_wav_files\AlexanderHenningDrachmann-20140108-koe\wav\b0457.wav True
voxforge_wav_files\AlexanderHenningDrachmann-20140108-koe\wav\b0463.wav True
voxforge_wav_files\AlexanderHenningDrachmann-20140108-koe\wav\b0465.wav True
voxforge_wav_files\AlexanderHenningDrachmann-20140108-koe\wav\b0466.wav True
voxforge_wav_files\AlexanderHenningDrachmann-20140108-koe\wav\b0464.wav True
voxforge_wav_files\AlexanderHenningDrachmann-20140108-koe\wav\b0460.wav True
voxforge_wav_files\AlexanderHenningDrachmann-20140108-koe\wav\b0462.wav True
voxforge_wav_files\AlexanderHenningDrachmann-20140108-mme\wav\a0230.wav True

voxforge_wav_files\AndreyBarrientosCostaRica-20130710-tpy\wav\b0211.wav True
voxforge_wav_files\AndreyBarrientosCostaRica-20130720-pxz\wav\a0431.wav True
voxforge_wav_files\AndreyBarrientosCostaRica-20130720-pxz\wav\a0428.wav True
voxforge_wav_files\AndreyBarrientosCostaRica-20130720-pxz\wav\a0437.wav True
voxforge_wav_files\AndreyBarrientosCostaRica-20130720-pxz\wav\a0436.wav True
voxforge_wav_files\AndreyBarrientosCostaRica-20130720-pxz\wav\a0434.wav True
voxforge_wav_files\AndreyBarrientosCostaRica-20130720-pxz\wav\a0433.wav True
voxforge_wav_files\AndreyBarrientosCostaRica-20130720-pxz\wav\a0429.wav True
voxforge_wav_files\AndreyBarrientosCostaRica-20130720-pxz\wav\a0435.wav True
voxforge_wav_files\AndreyBarrientosCostaRica-20130720-pxz\wav\a0432.wav True
voxforge_wav_files\AndreyBarrientosCostaRica-20130720-pxz\wav\a0430.wav True
voxforge_wav_files\Andy212380-20141016-lzl\wav\b0531.wav True
voxforge_wav_files\Andy212380-20141016-lzl\wav\b0532.wav True
voxforge_wav_files\Andy212380

voxforge_wav_files\Anniepoo-20140308-xot\wav\b0169.wav True
voxforge_wav_files\Anniepoo-20140308-xot\wav\b0171.wav True
voxforge_wav_files\Anniepoo-20140308-xot\wav\b0175.wav True
voxforge_wav_files\Anniepoo-20140308-xot\wav\b0177.wav True
voxforge_wav_files\Anniepoo-20140308-xot\wav\b0173.wav True
voxforge_wav_files\Anniepoo-20140308-xot\wav\b0176.wav True
voxforge_wav_files\Anniepoo-20140308-xot\wav\b0168.wav True
voxforge_wav_files\Anniepoo-20140308-zwy\wav\b0499.wav True
voxforge_wav_files\Anniepoo-20140308-zwy\wav\b0504.wav True
voxforge_wav_files\Anniepoo-20140308-zwy\wav\b0503.wav True
voxforge_wav_files\Anniepoo-20140308-zwy\wav\b0497.wav True
voxforge_wav_files\Anniepoo-20140308-zwy\wav\b0496.wav True
voxforge_wav_files\Anniepoo-20140308-zwy\wav\b0502.wav True
voxforge_wav_files\Anniepoo-20140308-zwy\wav\b0501.wav True
voxforge_wav_files\Anniepoo-20140308-zwy\wav\b0498.wav True
voxforge_wav_files\Anniepoo-20140308-zwy\wav\b0500.wav True
voxforge_wav_files\Anniepoo-20140308-zwy

...

voxforge_wav_files\cmu_us_ksp_arctic\wav\arctic_b0080.wav True
voxforge_wav_files\cmu_us_ksp_arctic\wav\arctic_b0459.wav True
voxforge_wav_files\cmu_us_ksp_arctic\wav\arctic_a0193.wav True
voxforge_wav_files\cmu_us_ksp_arctic\wav\arctic_a0433.wav True
voxforge_wav_files\cmu_us_ksp_arctic\wav\arctic_a0412.wav True
voxforge_wav_files\cmu_us_ksp_arctic\wav\arctic_b0021.wav True
voxforge_wav_files\cmu_us_ksp_arctic\wav\arctic_a0370.wav True
voxforge_wav_files\cmu_us_ksp_arctic\wav\arctic_b0246.wav True
voxforge_wav_files\cmu_us_ksp_arctic\wav\arctic_a0076.wav True
voxforge_wav_files\cmu_us_ksp_arctic\wav\arctic_b0521.wav True
voxforge_wav_files\cmu_us_ksp_arctic\wav\arctic_b0398.wav True
voxforge_wav_files\cmu_us_ksp_arctic\wav\arctic_a0033.wav True
voxforge_wav_files\cmu_us_ksp_arctic\wav\arctic_a0029.wav True
voxforge_wav_files\cmu_us_ksp_arctic\wav\arctic_a0584.wav True
voxforge_wav_files\cmu_us_ksp_arctic\wav\arctic_a0284.wav True
voxforge_wav_files\cmu_us_ksp_arctic\wav\arctic_a0173.w

voxforge_wav_files\cmu_us_ksp_arctic\wav\arctic_b0293.wav True
voxforge_wav_files\cmu_us_ksp_arctic\wav\arctic_b0057.wav True
voxforge_wav_files\cmu_us_ksp_arctic\wav\arctic_a0065.wav True
voxforge_wav_files\cmu_us_ksp_arctic\wav\arctic_b0087.wav True
voxforge_wav_files\cmu_us_ksp_arctic\wav\arctic_a0287.wav True
voxforge_wav_files\cmu_us_ksp_arctic\wav\arctic_a0540.wav True
voxforge_wav_files\cmu_us_ksp_arctic\wav\arctic_b0104.wav True
voxforge_wav_files\cmu_us_ksp_arctic\wav\arctic_a0534.wav True
voxforge_wav_files\cmu_us_ksp_arctic\wav\arctic_b0058.wav True
voxforge_wav_files\cmu_us_ksp_arctic\wav\arctic_b0331.wav True
voxforge_wav_files\cmu_us_ksp_arctic\wav\arctic_a0500.wav True
voxforge_wav_files\cmu_us_ksp_arctic\wav\arctic_b0404.wav True
voxforge_wav_files\cmu_us_ksp_arctic\wav\arctic_a0558.wav True
voxforge_wav_files\cmu_us_ksp_arctic\wav\arctic_a0139.wav True
voxforge_wav_files\cmu_us_ksp_arctic\wav\arctic_b0491.wav True
voxforge_wav_files\cmu_us_ksp_arctic\wav\arctic_a0276.w

voxforge_wav_files\cmu_us_ksp_arctic\wav\arctic_a0350.wav True
voxforge_wav_files\cmu_us_ksp_arctic\wav\arctic_b0383.wav True
voxforge_wav_files\cmu_us_ksp_arctic\wav\arctic_b0247.wav True
voxforge_wav_files\cmu_us_ksp_arctic\wav\arctic_b0310.wav True
voxforge_wav_files\cmu_us_ksp_arctic\wav\arctic_b0433.wav True
voxforge_wav_files\cmu_us_ksp_arctic\wav\arctic_b0377.wav True
voxforge_wav_files\cmu_us_ksp_arctic\wav\arctic_b0477.wav True
voxforge_wav_files\cmu_us_ksp_arctic\wav\arctic_b0500.wav True
voxforge_wav_files\cmu_us_ksp_arctic\wav\arctic_b0170.wav True
voxforge_wav_files\cmu_us_ksp_arctic\wav\arctic_b0151.wav True
voxforge_wav_files\cmu_us_ksp_arctic\wav\arctic_a0331.wav True
voxforge_wav_files\cmu_us_ksp_arctic\wav\arctic_b0163.wav True
voxforge_wav_files\cmu_us_ksp_arctic\wav\arctic_b0468.wav True
voxforge_wav_files\cmu_us_ksp_arctic\wav\arctic_b0336.wav True
voxforge_wav_files\cmu_us_ksp_arctic\wav\arctic_a0261.wav True
voxforge_wav_files\cmu_us_ksp_arctic\wav\arctic_b0140.w

voxforge_wav_files\cmu_us_ksp_arctic\wav\arctic_b0271.wav True
voxforge_wav_files\cmu_us_ksp_arctic\wav\arctic_b0063.wav True
voxforge_wav_files\cmu_us_ksp_arctic\wav\arctic_b0123.wav True
voxforge_wav_files\cmu_us_ksp_arctic\wav\arctic_a0220.wav True
voxforge_wav_files\cmu_us_ksp_arctic\wav\arctic_a0457.wav True
voxforge_wav_files\cmu_us_ksp_arctic\wav\arctic_a0291.wav True
voxforge_wav_files\cmu_us_ksp_arctic\wav\arctic_b0323.wav True
voxforge_wav_files\cmu_us_ksp_arctic\wav\arctic_a0390.wav True
voxforge_wav_files\cmu_us_ksp_arctic\wav\arctic_a0038.wav True
voxforge_wav_files\cmu_us_ksp_arctic\wav\arctic_b0511.wav True
voxforge_wav_files\cmu_us_ksp_arctic\wav\arctic_a0204.wav True
voxforge_wav_files\cmu_us_ksp_arctic\wav\arctic_a0573.wav True
voxforge_wav_files\cmu_us_ksp_arctic\wav\arctic_b0193.wav True
voxforge_wav_files\cmu_us_ksp_arctic\wav\arctic_b0465.wav True
voxforge_wav_files\cmu_us_ksp_arctic\wav\arctic_a0402.wav True
voxforge_wav_files\cmu_us_ksp_arctic\wav\arctic_a0236.w

KeyboardInterrupt: 

In [ ]:
X = np.array(X)
y = np.array(y)


In [ ]:
# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Build the MLP model
model = keras.Sequential([
    layers.Input(shape=(13,)),  # 13 MFCC features
    layers.Dense(64, activation="relu"),
    layers.Dense(32, activation="relu"),
    layers.Dense(1, activation="sigmoid")  # Binary classification
])

In [ ]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])


In [ ]:
# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.1)


In [9]:
# Evaluate
loss, acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {acc:.2f}")

NameError: name 'model' is not defined

In [10]:
model.save('model')

NameError: name 'model' is not defined

In [11]:
# Convert to DataFrame
feature_names = [f"mfcc_{i}" for i in range(13)]
X_df = pd.DataFrame(X, columns=feature_names)
X_df["gender"] = y  # Add labels

In [12]:
from pycaret.classification import *

# PyCaret Setup
clf_setup = setup(data=X_df, target="gender", train_size=0.8, session_id=42)

,Description,Value
0,Session id,42
1,Target,gender
2,Target type,Multiclass
3,Original data shape,"(46668, 14)"
4,Transformed data shape,"(46668, 14)"
5,Transformed train set shape,"(37334, 14)"
6,Transformed test set shape,"(9334, 14)"
7,Numeric features,13
8,Preprocess,True
9,Imputation type,simple


In [13]:
# Compare Models
best_model = compare_models()
print(f"Best Model: {best_model}")


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
et,Extra Trees Classifier,0.9038,0.9615,0.9038,0.9125,0.8848,0.7524,0.7730,0.4580
knn,K Neighbors Classifier,0.9033,0.9429,0.9033,0.8993,0.8993,0.7748,0.7762,0.3730
lightgbm,Light Gradient Boosting Machine,0.9022,0.9501,0.9022,0.9007,0.8885,0.7563,0.7678,0.7390
rf,Random Forest Classifier,0.8994,0.9514,0.8994,0.9077,0.8804,0.7405,0.7616,1.7000
gbc,Gradient Boosting Classifier,0.8633,0.0000,0.8633,0.8572,0.8361,0.6454,0.6660,9.2100
qda,Quadratic Discriminant Analysis,0.8536,0.0000,0.8536,0.8254,0.8295,0.6332,0.6447,0.0200
dt,Decision Tree Classifier,0.8192,0.7997,0.8192,0.8223,0.8206,0.5963,0.5965,0.1430
lr,Logistic Regression,0.7833,0.0000,0.7833,0.7395,0.7454,0.4062,0.4340,1.3820
lda,Linear Discriminant Analysis,0.7789,0.0000,0.7789,0.7317,0.7409,0.3972,0.4238,0.0160
nb,Naive Bayes,0.7613,0.8101,0.7613,0.7377,0.7409,0.4522,0.4560,0.0160


Best Model: ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='sqrt',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_samples_leaf=1,
                     min_samples_split=2, min_weight_fraction_leaf=0.0,
                     monotonic_cst=None, n_estimators=100, n_jobs=-1,
                     oob_score=False, random_state=42, verbose=0,
                     warm_start=False)


In [14]:
# Train Best Model
final_model = create_model(best_model)
final_model = finalize_model(final_model)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8998,0.9606,0.8998,0.9097,0.8800,0.7401,0.7630
1,0.8990,0.9565,0.8990,0.9077,0.8789,0.7405,0.7605
2,0.9076,0.9688,0.9076,0.9156,0.8887,0.7637,0.7826
3,0.9060,0.9667,0.9060,0.9148,0.8866,0.7582,0.7789
4,0.9084,0.9624,0.9084,0.9171,0.8906,0.7647,0.7850
5,0.9009,0.9623,0.9009,0.9099,0.8803,0.7442,0.7654
6,0.9065,0.9590,0.9065,0.9141,0.8900,0.7600,0.7797
7,0.9028,0.9568,0.9028,0.9114,0.8840,0.7499,0.7701
8,0.9038,0.9619,0.9038,0.9111,0.8853,0.7531,0.7728


In [15]:
# Evaluate
predictions = predict_model(final_model)
print(predictions.head())

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Extra Trees Classifier,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000


           mfcc_0      mfcc_1     mfcc_2     mfcc_3     mfcc_4     mfcc_5  \
8697  -474.597656   93.769928 -19.079102  33.896729  -9.355077   9.606043   
31536 -330.614014   67.306259   2.644576  38.657455   1.480155   9.448212   
6147  -311.142120   64.058624 -13.361362  41.694111 -17.610868  18.375080   
29311 -430.861084   75.929314 -20.579885   3.748899 -13.570605  -4.907367   
14027  -66.030647  105.483322 -23.584412  30.011114 -21.750191  16.163424   

          mfcc_6     mfcc_7     mfcc_8     mfcc_9   mfcc_10   mfcc_11  \
8697  -17.981239   5.611973 -17.078369   3.993233 -8.597243 -3.880171   
31536   7.870701  -0.584716  -3.364261   8.321804 -2.894572  5.336480   
6147   -1.034602 -10.461911   8.289107   4.478566  4.862948 -0.758509   
29311  -8.941398  -8.765680  -4.695103 -10.406656 -9.169727 -5.547805   
14027  -9.167974   9.567294 -16.414545  24.498062 -5.978817  7.436854   

         mfcc_12  gender  prediction_label  prediction_score  
8697  -12.733074       2           

In [16]:

save_model(final_model, 'my_final_best_model')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(exclude=None,
                                     include=['mfcc_0', 'mfcc_1', 'mfcc_2',
                                              'mfcc_3', 'mfcc_4', 'mfcc_5',
                                              'mfcc_6', 'mfcc_7', 'mfcc_8',
                                              'mfcc_9', 'mfcc_10', 'mfcc_11',
                                              'mfcc_12'],
                                     transformer=SimpleImputer(add_indicator=False,
                                                               copy=True,
                                                               fill_value=None,
                                                               keep_empty_features=False,
                                                               missing_values=nan,
                                                               strat...
                  Extra